```
p_паттерн парсит текст (str, pos), 
    вызывая другие паттерны, возвращающие древовидные структуры
    эти древовидные структуры передает одному из правил
        (остальные вызовы правил закомментированы 
            - в рамках одного текста каждый паттерн имеет ровно 1 смысл)
    и возвращает (pos, результат этого правила), помещенный в массив
    
    /*пока на ошибках парсинга концентрировать не будем*/
    если ничего не удалось распарсить, ???возвращаемый массив будет пустым???
    если удалось распарсить несколько вариантов - в массиве будет насколько вариантов
        сначала парсятся все исключения
        потом парсятся все обычные варианты (если нет исключений)
```

```
r_правило получает список древовидных структур
    обрабатывает их по определенному правилу
    возвращает древовидную структуру
```

```
древовидная структура - map, со следующими ключами
    type: 'noun'/'adj'/'verb'/'numeric'/...
    постоянные параметры (для сущ.: род, число)
    переменные параметры (для сущ.: падеж)
    talk: массив древовидных структур
        который также может содержать 1 слово (для сущ. в и.п.)
```

```
show древовидная структура
    превращает древовидную структуру в строку
```

```
для каждого типа элемента древовидной структуры будет map
    где каждой строке (для сущ. - слово в и.п.) будет соответствовать
        структура данных, которая вместе в переменными параметрами передается в 
        sh_функция для этого типа элемента древовидной структуры
            которая будет возвращать слово в соотв. форме (для сущ. в соотв. падеже)
```

# Общее

In [1]:
def throw(ex):
    raise ex

In [2]:
def en2ru(s): # main
    # добавить дочитывание точки и остаточных пробелов
    rezs=[res for pos,res in p_noun(s,0) if pos==len(s)]
    if len(rezs)>0: return [str(i) for i in rezs]
    rezs=[res for pos,res in p_adj(s,0) if pos==len(s)]
    if len(rezs)>0: return [str(i) for i in rezs]
    return []

## Паттерны парсинга

In [3]:
def p_spcs(str,pos):
    pos1=pos
    while pos1<len(str) and (str[pos1]==' ' or str[pos1]=='\n' or str[pos1]=='\t') :
        pos1+=1
    return [] if pos1==pos else [(pos1,'')]

In [4]:
def p_word(str,pos,word=None):
    if word!=None:
        return [(pos+len(word),word)] if str[pos:pos+len(word)]==word else []
    pos1=pos
    while pos1<len(str) and (str[pos1]>='a' and str[pos1]<='z' or \
                             str[pos1]>='A' and str[pos1]<='Z') :
        pos1+=1
    return [] if pos1==pos else [(pos1,str[pos:pos1])]

In [5]:
def p_seq(str,pos,*patterns):
    rezs=[]
    for patt in patterns:
        tmp=patt(str,pos)
        if len(tmp)>1 : raise NotImplementedError()
        if len(tmp)==0 : return []
        (pos,tmp)=tmp[0]
        rezs.append(tmp)
    return [(pos,rezs)]

# Словарь

## Классы отображения

In [6]:
class Struct:
    type=None
    talk=[] #массив структур или пар (строка-тип,строка-значение)
    def __str__(self):
        raise NotImplementedError('virtual function')

In [26]:
class StNoun(Struct):
    def __init__(self,odush=None,rod=None,chis=None,pad=None,word=None):
        self.type='noun'
        
        if odush==None and rod==None and chis==None and pad==None and word==None :
            self.odush=False
            self.rod='m'
            self.chis='ed'
            self.pad='ip'
            self.talk=[]
        else:
            if type(odush)!=bool : raise TypeError('odush must be bool')
            self.odush=odush

            if rod!='m' and rod!='g' and rod!='s' : raise TypeError('rod must be m or g or s')
            self.rod=rod

            if chis!='ed' and chis!='mn' : raise TypeError('chis must be ed or mn')
            self.chis=chis

            if pad!='ip' and pad!='rp' and pad!='dp' and pad!='vp' and pad!='tp' and pad!='pp' : 
                raise TypeError('pad must be ip, rp, dp, vp, tp or pp')
            self.pad=pad

            self.talk=[('main',word)]
        
    def __repr__(self):
        return '{type:'+self.type+',odush:'+str(self.odush)+',rod:'+self.rod+\
            ',chis:'+self.chis+',pad:'+self.pad+',talk:'+\
            repr(self.talk)+'}'
    
    def __str__(self):
        return ' '.join(\
            str(i) if isinstance(i,Struct) else \
            show_noun_map[i[1]](self) if i[0]=='main' else\
            i[1] if i[0]=='other' else\
            '#'+i[1] if i[0]=='punct' else throw(RuntimeError()) \
                        for i in self.talk).replace(' #','')

def show_noun1(pad,word,ip,rp,dp,vp,tp,pp):
    if pad=='ip' : return word+ip
    if pad=='rp' : return word+rp
    if pad=='dp' : return word+dp
    if pad=='vp' : return word+vp
    if pad=='tp' : return word+tp
    if pad=='pp' : return word+pp
    raise InternalError()


In [25]:
class StAdj(Struct):
    def __init__(self,odush=None,rod=None,chis=None,pad=None,word=None):
        self.type='adj'
        
        if odush==None and rod==None and chis==None and pad==None and word==None :
            self.odush=False
            self.rod='m'
            self.chis='ed'
            self.pad='ip'
            self.talk=[]
        else:
            if type(odush)!=bool : raise TypeError('odush must be bool')
            self.odush=odush

            if rod!='m' and rod!='g' and rod!='s' : raise TypeError('rod must be m or g or s')
            self.rod=rod

            if chis!='ed' and chis!='mn' : raise TypeError('chis must be ed or mn')
            self.chis=chis

            if pad!='ip' and pad!='rp' and pad!='dp' and pad!='vp' and pad!='tp'\
                    and pad!='pp'  and pad!='sh' : 
                raise TypeError('pad must be ip, rp, dp, vp, tp, pp or sh')
            self.pad=pad

            self.talk=[('main',word)]
        
    def __repr__(self):
        return '{type:'+self.type+',odush:'+str(self.odush)+',rod:'+self.rod+\
            ',chis:'+self.chis+',pad:'+self.pad+',talk:'+\
            repr(self.talk)+'}'
    
    def __str__(self):
        return ' '.join(\
            str(i) if isinstance(i,Struct) else \
            show_adj_map[i[1]](self) if i[0]=='main' else\
            i[1] if i[0]=='other' else\
            '#'+i[1] if i[0]=='punct' else throw(RuntimeError()) \
                        for i in self.talk).replace(' #','')

def show_adj1(st,word,ends):
    if st.chis=='mn' :
        (ip,rp,dp,vp,tp,pp,sh)=ends['mn']
    else:
        (ip,rp,dp,vp,tp,pp,sh)=ends[st.rod]
        
    if   st.pad=='ip' : rez=ip
    elif st.pad=='rp' : rez=rp
    elif st.pad=='dp' : rez=dp
    elif st.pad=='vp' : rez=vp[0] if st.odush else vp[1]
    elif st.pad=='tp' : rez=tp
    elif st.pad=='pp' : rez=pp
    elif st.pad=='sh' : rez=sh
    else: raise InternalError()
        
    return word+rez


## Отображение

In [9]:
show_adj_map={
    'летучий'  :lambda st: show_adj1(st,'летуч' ,
        {
            'm' :('ий','его','ему',('его','ий'),'им' ,'ем','' ),
            's' :('ее','его','ему',('ий', 'ий'),'им' ,'ем','е'),
            'g' :('ая','ей' ,'ей' ,('ую', 'ую'),'ей' ,'ей','а'),
            'mn':('ие','их' ,'им' ,('их', 'ие'),'ими','их','и'),
        }
    ) ,
}

In [51]:
show_noun_map={
}

## Правила

In [79]:
ruwords={}

def add_runoun(name,odush,rodchis,word,ip,rp,dp,vp,tp,pp):
    if rodchis=='mn':
        ruwords[name]=StNoun(odush,'m',rodchis,'ip',name)
    else:
        ruwords[name]=StNoun(odush,rodchis,'ed','ip',name)
    show_noun_map[name]=lambda st: show_noun1(st.pad,word,ip,rp,dp,vp,tp,pp)


In [80]:
#                                   ип ,нет,дать,вижу,творю,думаю
add_runoun("кот"   ,True ,'m','кот'  ,'' ,'а','у' ,'а' ,'ом' ,'е' )

add_runoun("урок"  ,False,'m','урок' ,'' ,'а','у' ,''  ,'ом' ,'е' )

add_runoun("кошка" ,True ,'g','кошк' ,'а','и','е' ,'у' ,'ой' ,'е' )
add_runoun("кепка" ,False,'g','кепк' ,'а','и','е' ,'у' ,'ой' ,'е' )
add_runoun("шапка" ,False,'g','шапк' ,'а','и','е' ,'у' ,'ой' ,'е' )
add_runoun("ручка" ,False,'g','ручк' ,'а','и','е' ,'у' ,'ой' ,'е' )

add_runoun("крыса" ,True ,'g','крыс' ,'а','ы','е' ,'у' ,'ой' ,'е' )
add_runoun("шляпа" ,False,'g','шляп' ,'а','ы','е' ,'у' ,'ой' ,'е' )
add_runoun("курица",False,'g','куриц','а','ы','е' ,'у' ,'ой' ,'е' )

add_runoun("мышь"  ,True ,'g','мыш' ,'ь','и','и' ,'ь' ,'ью' ,'и' )


In [12]:
def r_letuchij(): return StAdj(True,'g','ed','ip','летучий')

In [94]:
def r_a_noun(a,n): return n

def r_adj_noun(aa,nn): 
    from copy import copy
    a=copy(aa)
    n=copy(nn)
    n.talk=copy(n.talk)
    a.rod=n.rod
    a.odush=n.odush
    n.talk.insert(0,a)
    return n

def r_snoun_and_noun(sn,n):
    root=StNoun()
    root.chis='mn'
    root.talk=[sn,('other','и'),n]
    return root
def r_snoun_comma_noun(sn,n):
    root=StNoun()
    root.chis='mn'
    root.talk=[sn,('punct',','),n]
    return root

## Словарь

In [29]:
dict_adj={
    'a' : '',
    #'a' : r_nekotoryj(),
    'an' : '',
    #'an' : r_nekotoryj(),
}

In [83]:
dict_noun={
    'cat':   ruwords["кошка"],
    #'cat':   ruwords["кот"],
    
    #'rat':   ruwords["мышь"],
    'rat':   ruwords['крыса'],
    
    'bat':   r_adj_noun(r_letuchij(),ruwords["мышь"]),
    
    'lesson':ruwords["урок"],
    
    'cap':   ruwords["кепка"],
    #'cap':  ruwords["шапка"],
    
    'pen':   ruwords["ручка"],
    
    'hat':   ruwords["шляпа"],
    
    'hen':   ruwords["курица"],
}

## Паттерны

In [59]:
def p_adj(str,pos):
    rezs=[]
    for tpos,tword in p_word(str,pos):
        if tword in dict_adj:
            rezs.append((tpos,dict_adj[tword]))
    return rezs

In [60]:
def p_adj_noun(str,pos):
    rezs=[]
    #исключение словосочетания
    if 1:
        def lp_an_a(str,pos):
            lrezs=p_word(str,pos,'an')
            if len(lrezs)>0 : return lrezs
            return p_word(str,pos,'a')
        rezs+=[\
            (pos1,r_a_noun(a,n)) \
            #(pos1,r_nekotoryj_noun(a,s)) \
                for (pos1,[a,s,n]) in p_seq(str,pos,lp_an_a,p_spcs,p_noun) ]
    
    if len(rezs)>0 : return rezs
    #словосочетание
    return [\
        (pos1,r_adj_noun(a,n)) \
        #(pos1,r_nekotoryj_noun(a,s)) \
            for (pos1,[a,s,n]) in p_seq(str,pos,p_adj,p_spcs,p_noun) ]
    

In [61]:
def p_simple_noun(str,pos):
    rezs=p_adj_noun(str,pos)
    if len(rezs)>0 : return rezs

    #слово
    tmp=[]
    for tpos,tword in p_word(str,pos):
        if tword in dict_noun:
            tmp.append((tpos,dict_noun[tword]))
    rezs+=tmp
        
    return rezs

In [62]:
def p_snoun_and_noun(str,pos):
    def lp_and(str,pos): return p_word(str,pos,'and')
    def lp_comma(str,pos): return p_word(str,pos,',')
    rezs= [\
        (pos1,r_snoun_and_noun(sn,n)) \
        #(pos1,r_nekotoryj_noun(a,s)) \
            for (pos1,[sn,s1,a,s2,n]) in p_seq(str,pos,\
                p_simple_noun,p_spcs,lp_and,p_spcs,p_noun) ]
    if len(rezs)>0: return rezs
    
    return [\
        (pos1,r_snoun_comma_noun(sn,n)) \
        #(pos1,r_nekotoryj_noun(a,s)) \
            for (pos1,[sn,c,s2,n]) in p_seq(str,pos,\
                p_simple_noun,lp_comma,p_spcs,p_noun) ]
    

In [63]:
def p_noun(str,pos):
    rezs=p_snoun_and_noun(str,pos)
    if len(rezs)>0 : return rezs
    return p_simple_noun(str,pos)

# Тесты

```
getter & setter падежей сущ
числительные: род число падеж одуш - пока будем использовать как прилагательные, 
    но число числит. и число сущ. должны совпадать - проверка во время парсинга, 
        иначе исключение
```

In [64]:
r_koshka()

{type:noun,odush:True,rod:g,chis:ed,pad:ip,talk:[('main', 'кошка')]}

In [65]:
p_noun('cat',0)

[(3, {type:noun,odush:True,rod:g,chis:ed,pad:ip,talk:[('main', 'кошка')]})]

In [66]:
p_spcs(' ',0)

[(1, '')]

In [67]:
def foo():
    x=1
    def bar():
        nonlocal x
        x=2
    bar()
    print(x)
foo()

2


In [68]:
StNoun()

{type:noun,odush:False,rod:m,chis:ed,pad:ip,talk:[]}

## Lesson 1

In [69]:
en2ru('cat')

['кошка']

In [70]:
en2ru('a cat')

['кошка']

In [71]:
en2ru('rat')

['крыса']

In [85]:
en2ru('bat')

['летучая мышь']

In [86]:
en2ru('a')

['']

In [87]:
en2ru('lesson')

['урок']

In [88]:
en2ru('a rat, a bat, a cat')

['крыса, летучая мышь, кошка']

## Lesson 2

In [91]:
en2ru('a pen and a hen')

['ручка и курица']

In [92]:
en2ru('a bat and a cat')

['летучая мышь и кошка']

In [93]:
en2ru('a hen, a bat, a cat and a rat')

['курица, летучая мышь, кошка и крыса']